In [8]:
import pandas as pd
import numpy as np
import torch
from collections import Counter
import random
import scipy.stats
from sklearn.preprocessing import MinMaxScaler
import joblib

In [9]:
"""labels:
0 - other
1 - head non-standard plane
2 - head standard plane
3 - femur non-standard plane
4 - femur standard plane
5 - femur non standard plane
6 - femur standard plane"""

'labels:\n0 - other\n1 - head non-standard plane\n2 - head standard plane\n3 - femur non-standard plane\n4 - femur standard plane\n5 - femur non standard plane\n6 - femur standard plane'

In [10]:
train = pd.read_csv('/data/kpusteln/Fetal-RL/data_preparation/data_biometry/ete_model/biometry_scaled_ps/biometry_train_scaled_size.csv')
val = pd.read_csv('/data/kpusteln/Fetal-RL/data_preparation/data_biometry/ete_model/biometry_scaled_ps/biometry_val_scaled_size.csv')
test = pd.read_csv('/data/kpusteln/Fetal-RL/data_preparation/data_biometry/ete_model/biometry_scaled_ps/biometry_test_scaled_size.csv')

In [11]:
train_org = pd.read_csv('/data/kpusteln/Fetal-RL/data_preparation/data_biometry/ete_model/biometry_train.csv')
test_org = pd.read_csv('/data/kpusteln/Fetal-RL/data_preparation/data_biometry/ete_model/biometry_test.csv')

val_org = pd.read_csv('/data/kpusteln/Fetal-RL/data_preparation/data_biometry/ete_model/biometry_val.csv')

In [12]:
train['days'] = train_org['days']
test['days'] = test_org['days']
val['days'] = val_org['days']

In [13]:
train = pd.concat([train, test])

In [14]:
train_head = train[train['Class'] == 2]
train_abdomen = train[train['Class'] == 4]
train_femur = train[train['Class'] == 6]

val_head = val[val['Class'] == 2]
val_abdomen = val[val['Class'] == 4]
val_femur = val[val['Class'] == 6]

#test_head = test[test['Class'] == 2]
#test_abdomen = test[test['Class'] == 4]
#test_femur = test[test['Class'] == 6]

In [19]:
classes = train_abdomen['video'].str.split('_', expand=True)[1]

In [20]:
classes.value_counts()

2    3625
3      26
Name: 1, dtype: int64

In [21]:
train_abdomen_filtered = train_abdomen[classes == '2']
#test_abdomen_filtered = test_abdomen[test_abdomen['video'].str.split('_', expand=True)[1] == '2']
val_abdomen_filtered = val_abdomen[val_abdomen['video'].str.split('_', expand=True)[1] == '2']

In [22]:
train_head_filtered = train_head[train_head['video'].str.split('_', expand=True)[1] == '1']
#test_head_filtered = test_head[test_head['video'].str.split('_', expand=True)[1] == '1']
val_head_filtered = val_head[val_head['video'].str.split('_', expand=True)[1] == '1']

In [23]:
train_femur_filtered = train_femur[train_femur['video'].str.split('_', expand=True)[1] == '3']
#test_femur_filtered = test_femur[test_femur['video'].str.split('_', expand=True)[1] == '3']
val_femur_filtered = val_femur[val_femur['video'].str.split('_', expand=True)[1] == '3']

In [24]:
train_abdomen_filtered

,index,Class,video,measures,ps,frames_n,measure_scaled,days,frame_loc,height,width,height_org,width_org,measure_normalized
226,6_2_35,4,6_2,16.47,0.120130,51,0.598042,147,0.686275,86.49360,115.32480,720,960,0.457089
227,6_2_36,4,6_2,16.47,0.120130,51,0.598042,147,0.705882,86.49360,115.32480,720,960,0.457089
228,6_2_37,4,6_2,16.47,0.120130,51,0.598042,147,0.725490,86.49360,115.32480,720,960,0.457089
229,6_2_38,4,6_2,16.47,0.120130,51,0.598042,147,0.745098,86.49360,115.32480,720,960,0.457089
230,6_2_39,4,6_2,16.47,0.120130,51,0.598042,147,0.764706,86.49360,115.32480,720,960,0.457089
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19374,669_2_46,4,669_2,22.90,0.170650,47,0.584303,171,0.978723,122.86800,163.82400,720,960,0.659227
19375,669_2_47,4,669_2,22.90,0.170650,47,0.584303,171,1.000000,122.86800,163.82400,720,960,0.659227
19555,675_2_56,4,675_2,31.87,0.199092,61,0.706560,257,0.918033,143.34624,191.12832,720,960,0.941213
19556,675_2_57,4,675_2,31.87,0.199092,61,0.706560,257,0.934426,143.34624,191.12832,720,960,0.941213


In [25]:
train_abdomen_filtered['measure_scaled'] = train_abdomen_filtered['measures']/train_abdomen_filtered['ps']
#test_abdomen_filtered['measure_scaled'] = test_abdomen_filtered['measures']/test_abdomen_filtered['ps']
val_abdomen_filtered['measure_scaled'] = val_abdomen_filtered['measures']/val_abdomen_filtered['ps']

/data/kpusteln/my-envs/swin/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/data/kpusteln/my-envs/swin/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [26]:
scaler = MinMaxScaler()
measure_scaled = train_abdomen_filtered['measure_scaled']
scaler.fit(measure_scaled.values.reshape(-1,1))

MinMaxScaler()

In [27]:
train_abdomen_filtered['measure_scaled'] = scaler.transform(train_abdomen_filtered['measure_scaled'].values.reshape(-1,1))
#test_abdomen_filtered['measure_scaled'] = scaler.transform(test_abdomen_filtered['measure_scaled'].values.reshape(-1,1))
val_abdomen_filtered['measure_scaled'] = scaler.transform(val_abdomen_filtered['measure_scaled'].values.reshape(-1,1))

/data/kpusteln/my-envs/swin/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/data/kpusteln/my-envs/swin/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [28]:
scaler.inverse_transform(train_abdomen_filtered['measure_scaled'].values.reshape(-1,1))

array([[137.1014734 ],
       [137.1014734 ],
       [137.1014734 ],
       ...,
       [160.07674844],
       [160.07674844],
       [160.07674844]])

In [29]:
joblib.dump(scaler, 'abdomen_scaler')

['abdomen_scaler']

In [30]:
train_femur_filtered['measure_scaled'] = train_femur_filtered['measures']/train_femur_filtered['ps']
#test_femur_filtered['measure_scaled'] = test_femur_filtered['measures']/test_femur_filtered['ps']
val_femur_filtered['measure_scaled'] = val_femur_filtered['measures']/val_femur_filtered['ps']

/data/kpusteln/my-envs/swin/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [31]:
scaler = MinMaxScaler()
measure_scaled = train_femur_filtered['measure_scaled']
scaler.fit(measure_scaled.values.reshape(-1,1))

MinMaxScaler()

In [32]:
train_femur_filtered['measure_scaled'] = scaler.transform(train_femur_filtered['measure_scaled'].values.reshape(-1,1))
#test_femur_filtered['measure_scaled'] = scaler.transform(test_femur_filtered['measure_scaled'].values.reshape(-1,1))
val_femur_filtered['measure_scaled'] = scaler.transform(val_femur_filtered['measure_scaled'].values.reshape(-1,1))

/data/kpusteln/my-envs/swin/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [33]:
scaler.inverse_transform(train_femur_filtered['measure_scaled'].values.reshape(-1,1))

array([[47.37117519],
       [47.37117519],
       [47.37117519],
       ...,
       [21.68494119],
       [21.68494119],
       [36.72128331]])

In [34]:
joblib.dump(scaler, 'femur_scaler')

['femur_scaler']

In [35]:
train_head_filtered['measure_scaled'] = train_head_filtered['measures']/train_head_filtered['ps']
#test_head_filtered['measure_scaled'] = test_head_filtered['measures']/test_head_filtered['ps']
val_head_filtered['measure_scaled'] = val_head_filtered['measures']/val_head_filtered['ps']

/data/kpusteln/my-envs/swin/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/data/kpusteln/my-envs/swin/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [36]:
scaler = MinMaxScaler()
measure_scaled = train_head_filtered['measure_scaled']
scaler.fit(measure_scaled.values.reshape(-1,1))

MinMaxScaler()

In [37]:
train_head_filtered['measure_scaled'] = scaler.transform(train_head_filtered['measure_scaled'].values.reshape(-1,1))
#test_head_filtered['measure_scaled'] = scaler.transform(test_head_filtered['measure_scaled'].values.reshape(-1,1))
val_head_filtered['measure_scaled'] = scaler.transform(val_head_filtered['measure_scaled'].values.reshape(-1,1))

/data/kpusteln/my-envs/swin/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/data/kpusteln/my-envs/swin/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [38]:
scaler.inverse_transform(train_head_filtered['measure_scaled'].values.reshape(-1,1))

array([[149.17173063],
       [149.17173063],
       [149.17173063],
       ...,
       [182.71523118],
       [182.71523118],
       [182.71523118]])

In [39]:
joblib.dump(scaler, 'head_scaler')

['head_scaler']

In [40]:
train_abdomen_filtered.to_csv('/data/kpusteln/Fetal-RL/data_preparation/data_biometry/ete_model/biometry_scaled_ps/abdomen/biometry_train_scaled_size_abdomen.csv', index=False)
val_abdomen_filtered.to_csv('/data/kpusteln/Fetal-RL/data_preparation/data_biometry/ete_model/biometry_scaled_ps/abdomen/biometry_val_scaled_size_abdomen.csv', index=False)
#test_abdomen_filtered.to_csv('/data/kpusteln/Fetal-RL/data_preparation/data_biometry/ete_model/biometry_scaled_ps/abdomen/biometry_test_scaled_size_abdomen.csv', index=False)

In [41]:
train_femur_filtered.to_csv('/data/kpusteln/Fetal-RL/data_preparation/data_biometry/ete_model/biometry_scaled_ps/femur/biometry_train_scaled_size_femur.csv', index=False)
#test_femur_filtered.to_csv('/data/kpusteln/Fetal-RL/data_preparation/data_biometry/ete_model/biometry_scaled_ps/femur/biometry_test_scaled_size_femur.csv', index=False)
val_femur_filtered.to_csv('/data/kpusteln/Fetal-RL/data_preparation/data_biometry/ete_model/biometry_scaled_ps/femur/biometry_val_scaled_size_femur.csv', index=False)



In [42]:
train_head_filtered.to_csv('/data/kpusteln/Fetal-RL/data_preparation/data_biometry/ete_model/biometry_scaled_ps/head/biometry_train_scaled_size_head.csv', index=False)
#test_head_filtered.to_csv('/data/kpusteln/Fetal-RL/data_preparation/data_biometry/ete_model/biometry_scaled_ps/head/biometry_test_scaled_size_head.csv', index=False)
val_head_filtered.to_csv('/data/kpusteln/Fetal-RL/data_preparation/data_biometry/ete_model/biometry_scaled_ps/head/biometry_val_scaled_size_head.csv', index=False)

In [43]:
train_all = pd.concat([train_abdomen_filtered, train_femur_filtered, train_head_filtered])
#test_all = pd.concat([test_abdomen_filtered, test_femur_filtered, test_head_filtered])
val_all = pd.concat([val_abdomen_filtered, val_femur_filtered, val_head_filtered])

In [44]:
train_all['measure_scaled'] = train_all['measures']/train_all['ps']
#test_all['measure_scaled'] = test_all['measures']/test_all['ps']
val_all['measure_scaled'] = val_all['measures']/val_all['ps']

In [45]:
train_all

,index,Class,video,measures,ps,frames_n,measure_scaled,days,frame_loc,height,width,height_org,width_org,measure_normalized
226,6_2_35,4,6_2,16.47,0.120130,51,137.101473,147,0.686275,86.49360,115.32480,720,960,0.457089
227,6_2_36,4,6_2,16.47,0.120130,51,137.101473,147,0.705882,86.49360,115.32480,720,960,0.457089
228,6_2_37,4,6_2,16.47,0.120130,51,137.101473,147,0.725490,86.49360,115.32480,720,960,0.457089
229,6_2_38,4,6_2,16.47,0.120130,51,137.101473,147,0.745098,86.49360,115.32480,720,960,0.457089
230,6_2_39,4,6_2,16.47,0.120130,51,137.101473,147,0.764706,86.49360,115.32480,720,960,0.457089
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3121,117_1_40,2,117_1,19.85,0.108639,44,182.715231,161,0.909091,78.22008,104.29344,720,960,0.563345
3122,117_1_41,2,117_1,19.85,0.108639,44,182.715231,161,0.931818,78.22008,104.29344,720,960,0.563345
3123,117_1_42,2,117_1,19.85,0.108639,44,182.715231,161,0.954545,78.22008,104.29344,720,960,0.563345
3124,117_1_43,2,117_1,19.85,0.108639,44,182.715231,161,0.977273,78.22008,104.29344,720,960,0.563345


In [46]:
scaler = MinMaxScaler()
measure_scaled = train_all['measure_scaled']
scaler.fit(measure_scaled.values.reshape(-1,1))

MinMaxScaler()

In [47]:
train_all['measure_scaled'] = scaler.transform(train_all['measure_scaled'].values.reshape(-1,1))
#test_all['measure_scaled'] = scaler.transform(test_all['measure_scaled'].values.reshape(-1,1))
val_all['measure_scaled'] = scaler.transform(val_all['measure_scaled'].values.reshape(-1,1))

In [48]:
train_all

,index,Class,video,measures,ps,frames_n,measure_scaled,days,frame_loc,height,width,height_org,width_org,measure_normalized
226,6_2_35,4,6_2,16.47,0.120130,51,0.618863,147,0.686275,86.49360,115.32480,720,960,0.457089
227,6_2_36,4,6_2,16.47,0.120130,51,0.618863,147,0.705882,86.49360,115.32480,720,960,0.457089
228,6_2_37,4,6_2,16.47,0.120130,51,0.618863,147,0.725490,86.49360,115.32480,720,960,0.457089
229,6_2_38,4,6_2,16.47,0.120130,51,0.618863,147,0.745098,86.49360,115.32480,720,960,0.457089
230,6_2_39,4,6_2,16.47,0.120130,51,0.618863,147,0.764706,86.49360,115.32480,720,960,0.457089
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3121,117_1_40,2,117_1,19.85,0.108639,44,0.841410,161,0.909091,78.22008,104.29344,720,960,0.563345
3122,117_1_41,2,117_1,19.85,0.108639,44,0.841410,161,0.931818,78.22008,104.29344,720,960,0.563345
3123,117_1_42,2,117_1,19.85,0.108639,44,0.841410,161,0.954545,78.22008,104.29344,720,960,0.563345
3124,117_1_43,2,117_1,19.85,0.108639,44,0.841410,161,0.977273,78.22008,104.29344,720,960,0.563345


In [49]:
train_all.to_csv('/data/kpusteln/Fetal-RL/data_preparation/data_biometry/ete_model/biometry_scaled_ps/all/biometry_train_scaled_size_all.csv', index=False)
#test_all.to_csv('/data/kpusteln/Fetal-RL/data_preparation/data_biometry/ete_model/biometry_scaled_ps/all/biometry_test_scaled_size_all.csv', index=False)
val_all.to_csv('/data/kpusteln/Fetal-RL/data_preparation/data_biometry/ete_model/biometry_scaled_ps/all/biometry_val_scaled_size_all.csv', index=False)

In [50]:
joblib.dump(scaler, 'all_scaler')

['all_scaler']

In [51]:
scaler = joblib.load('all_scaler')

In [52]:
train_all

,index,Class,video,measures,ps,frames_n,measure_scaled,days,frame_loc,height,width,height_org,width_org,measure_normalized
226,6_2_35,4,6_2,16.47,0.120130,51,0.618863,147,0.686275,86.49360,115.32480,720,960,0.457089
227,6_2_36,4,6_2,16.47,0.120130,51,0.618863,147,0.705882,86.49360,115.32480,720,960,0.457089
228,6_2_37,4,6_2,16.47,0.120130,51,0.618863,147,0.725490,86.49360,115.32480,720,960,0.457089
229,6_2_38,4,6_2,16.47,0.120130,51,0.618863,147,0.745098,86.49360,115.32480,720,960,0.457089
230,6_2_39,4,6_2,16.47,0.120130,51,0.618863,147,0.764706,86.49360,115.32480,720,960,0.457089
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3121,117_1_40,2,117_1,19.85,0.108639,44,0.841410,161,0.909091,78.22008,104.29344,720,960,0.563345
3122,117_1_41,2,117_1,19.85,0.108639,44,0.841410,161,0.931818,78.22008,104.29344,720,960,0.563345
3123,117_1_42,2,117_1,19.85,0.108639,44,0.841410,161,0.954545,78.22008,104.29344,720,960,0.563345
3124,117_1_43,2,117_1,19.85,0.108639,44,0.841410,161,0.977273,78.22008,104.29344,720,960,0.563345


In [29]:
softmax = torch.nn.Softmax()
softmax(torch.tensor([0.1, 0.2, 0.3, float('-inf')]))

/data/kpusteln/my-envs/swin/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  


tensor([0.3006, 0.3322, 0.3672, 0.0000])

In [55]:
train = pd.read_csv('/data/kpusteln/Fetal-RL/data_preparation/data_biometry/ete_model/biometry_scaled_ps/abdomen/biometry_train_scaled_size_abdomen.csv')
val = pd.read_csv('/data/kpusteln/Fetal-RL/data_preparation/data_biometry/ete_model/biometry_scaled_ps/abdomen/biometry_val_scaled_size_abdomen.csv')

In [57]:
videos_train = train['video'].unique()
videos_val = val['video'].unique()
videos_train = pd.DataFrame(videos_train, columns=['video'])
videos_val = pd.DataFrame(videos_val, columns=['video'])

In [58]:
videos_train.to_csv('/data/kpusteln/Fetal-RL/data_preparation/data_biometry/ete_model/biometry_scaled_ps/abdomen/abd_videos_train.csv', index=False)
videos_val.to_csv('/data/kpusteln/Fetal-RL/data_preparation/data_biometry/ete_model/biometry_scaled_ps/abdomen/abd_videos_val.csv', index=False)